# Profiling BatchFlow code

A profile is a set of statistics that describes how often and for how long various parts of the program executed.

This notebooks shows how to profile various parts of BatchFlow: namely, pipelines and models.

In [1]:
import sys

sys.path.append("../../..")
from batchflow import B, V, W
from batchflow.opensets import MNIST
from batchflow.models.torch import ResNet18

In [2]:
dataset = MNIST()

To collect information about model training times (both on CPU and GPU), one must set `profile` option in the model configuration to `True`:

In [3]:
model_config = {
    'inputs/labels/classes': 10,
    'loss': 'ce',
    'profile': True,
}

In [4]:
pipeline = (dataset.train.p
                .init_variable('loss_history', [])
                .to_array(channels='first', dtype='float32')
                .multiply(multiplier=1/255., preserve_type=False)
                .init_model('dynamic', ResNet18,
                            'resnet', config=model_config)
                .train_model('resnet',
                             B.images, B.labels,
                             fetches='loss',
                             save_to=V('loss_history', mode='a'))
                )

To gather statistics about how long each action takes, we must set `profile` to `True` inside `run` call:

In [5]:
BATCH_SIZE = 64
N_ITERS = 50

pipeline.run(BATCH_SIZE, n_iters=N_ITERS, bar=True, profile=True,
                   bar_desc=W(V('loss_history')[-1].format('Loss is {:7.7}')))

Loss is 0.1426592: 100%|██████████| 50/50 [01:22<00:00,  1.29s/it] 


# Pipeline profiling

First of all, there is an `elapsed_time` attribute inside every instance of `Pipeline`: it stores total time of running the pipeline (even if it was used multiple times):

In [6]:
pipeline.elapsed_time

82.73438310623169

Note that `elapsed_time` attribute is created whether or not we set `profile` to `True`.

After running with `profile=True`, pipeline has attribute `profile_info`: this `DataFrame` holds collected information:

In [7]:
pipeline.profile_info.head()

iter  \
action      id                                                         
to_array #0 <built-in method _abc._abc_instancecheck>::/hom...     1   
            <built-in method _abc._abc_subclasscheck>::/hom...     1   
            <built-in method _operator.index>::/home/tsimfe...     1   
            <built-in method posix.sched_getaffinity>::../....     1   
            <built-in method builtins.any>::../../../batchf...     1   

                                                                total_time  \
action      id                                                               
to_array #0 <built-in method _abc._abc_instancecheck>::/hom...    0.081367   
            <built-in method _abc._abc_subclasscheck>::/hom...    0.081367   
            <built-in method _operator.index>::/home/tsimfe...    0.081367   
            <built-in method posix.sched_getaffinity>::../....    0.081367   
            <built-in method builtins.any>::../../../batchf...    0.081367   

                                                                pipeline_time  \
action      id                                                                  
to_array #0 <built-in method _abc._abc_instancecheck>::/hom...         0.0711   
            <built-in method _abc._abc_subclasscheck>::/hom...         0.0711   
            <built-in method _operator.index>::/home/tsimfe...         0.0711   
            <built-in method posix.sched_getaffinity>::../....         0.0711   
            <built-in method builtins.any>::../../../batchf...         0.0711   

                                                                ncalls  \
action      id                                                           
to_array #0 <built-in method _abc._abc_instancecheck>::/hom...      64   
            <built-in method _abc._abc_subclasscheck>::/hom...      57   
            <built-in method _operator.index>::/home/tsimfe...     128   
            <built-in method posix.sched_getaffinity>::../....       1   
            <built-in method builtins.any>::../../../batchf...       1   

                                                                 tottime  \
action      id                                                             
to_array #0 <built-in method _abc._abc_instancecheck>::/hom...  0.000166   
            <built-in method _abc._abc_subclasscheck>::/hom...  0.001096   
            <built-in method _operator.index>::/home/tsimfe...  0.000037   
            <built-in method posix.sched_getaffinity>::../....  0.000048   
            <built-in method builtins.any>::../../../batchf...  0.000003   

                                                                 cumtime  \
action      id                                                             
to_array #0 <built-in method _abc._abc_instancecheck>::/hom...  0.001459   
            <built-in method _abc._abc_subclasscheck>::/hom...  0.001288   
            <built-in method _operator.index>::/home/tsimfe...  0.000037   
            <built-in method posix.sched_getaffinity>::../....  0.000048   
            <built-in method builtins.any>::../../../batchf...  0.000026   

                                                                       batch_id  \
action      id                                                                    
to_array #0 <built-in method _abc._abc_instancecheck>::/hom...  139994263066552   
            <built-in method _abc._abc_subclasscheck>::/hom...  139994263066552   
            <built-in method _operator.index>::/home/tsimfe...  139994263066552   
            <built-in method posix.sched_getaffinity>::../....  139994263066552   
            <built-in method builtins.any>::../../../batchf...  139994263066552   

                                                                  start_time  
action      id                                                                
to_array #0 <built-in method _abc._abc_instancecheck>::/hom...  1.582270e+09  
            <built-in method _abc._abc_s

Note that there is a detailed information about exact methods that are called inside each of the actions. That is a lot of data which can give us precise understanding of parts of the code, that are our bottlenecks.

Columns of the `profile_info`:
- `action`, `iter`, `batch_id` and `start_time` are pretty self-explainable
- `id` allows to identify exact method with great details: it is a concatenation of `method_name`, `file_name`, `line_number` and `callee`
- `total_time` is a time taken by an action
- `pipeline_time` is `total_time` plus time of processing the profiling table at each iteration
- `tottime` is a time taken by a method inside action
- `cumtime` is a time take by a method and all of the methods that are called inside this method


More often than not, though, we don't need such granularity. Pipeline method `show_profile_info` makes some handy aggregations:

**Note:** by default, results are sorted on `total_time` or `tottime`, depending on level of details.

In [8]:
# timings for each action
pipeline.show_profile_info(per_iter=False, detailed=False)

total_time                     pipeline_time            \
                      sum      mean       max           sum      mean   
action                                                                  
train_model #2  75.821759  1.516435  3.217391     75.503571  1.510071   
to_array #0      2.835696  0.056714  0.106486      2.546398  0.050928   
multiply #1      1.692740  0.033855  0.069450      1.509747  0.030195   

                          
                     max  
action                    
train_model #2  3.205411  
to_array #0     0.097141  
multiply #1     0.060036

In [9]:
# for each action show 2 of the slowest methods, based on maximum `ncalls`
pipeline.show_profile_info(per_iter=False, detailed=True, sortby=('ncalls', 'max'), limit=2)

ncalls  \
                                                                       sum   
action         id                                                            
multiply #1    <built-in method builtins.isinstance>::../../.....   409600   
               <built-in method numpy.where>::../../../batchfl...   204800   
to_array #0    <built-in method builtins.isinstance>::../../.....   409600   
               <built-in method numpy.where>::../../../batchfl...   204800   
train_model #2 <method 'append' of 'list' objects>::/home/tsim...  1082936   
               <lambda>::~::0::<method 'sort' of 'list' objects>    541468   

                                                                             \
                                                                       mean   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....   8192.00   
               <built-in method numpy.where>::../../../batchfl...   4096.00   
to_array #0    <built-in method builtins.isinstance>::../../.....   8192.00   
               <built-in method numpy.where>::../../../batchfl...   4096.00   
train_model #2 <method 'append' of 'list' objects>::/home/tsim...  21658.72   
               <lambda>::~::0::<method 'sort' of 'list' objects>   10829.36   

                                                                          \
                                                                     max   
action         id                                                          
multiply #1    <built-in method builtins.isinstance>::../../.....   8192   
               <built-in method numpy.where>::../../../batchfl...   4096   
to_array #0    <built-in method builtins.isinstance>::../../.....   8192   
               <built-in method numpy.where>::../../../batchfl...   4096   
train_model #2 <method 'append' of 'list' objects>::/home/tsim...  21686   
               <lambda>::~::0::<method 'sort' of 'list' objects>   10843   

                                                                    tottime  \
                                                                        sum   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....  0.050672   
               <built-in method numpy.where>::../../../batchfl...  0.111507   
to_array #0    <built-in method builtins.isinstance>::../../.....  0.069341   
               <built-in method numpy.where>::../../../batchfl...  0.167002   
train_model #2 <method 'append' of 'list' objects>::/home/tsim...  0.151943   
               <lambda>::~::0::<method 'sort' of 'list' objects>   0.091438   

                                                                             \
                                                                       mean   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....  0.001013   
               <built-in method numpy.where>::../../../batchfl...  0.002230   
to_array #0    <built-in method builtins.isinstance>::../../.....  0.001387   
               <built-in method numpy.where>::../../../batchfl...  0.003340   
train_model #2 <method 'append' of 'list' objects>::/home/tsim...  0.003039   
               <lambda>::~::0::<method 'sort' of 'list' objects>   0.001829   

                                                                             \
                                                                        max   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....  0.001615   
               <built-in method numpy.where>::../../../batchfl...  0.003821   
to_array #0    <built-in method builtins.isinstance>::../../.....  0.002048   
               <built-in method numpy.where>::../../../batchfl...  0.0

In [10]:
# timings for each action for each iter
pipeline.show_profile_info(per_iter=True, detailed=False,)

total_time  pipeline_time         batch_id
iter action                                                    
1    train_model #2    3.217391       3.205411  139994263066555
     to_array #0       0.081367       0.071100  139994263066552
     multiply #1       0.024790       0.021719  139994263066552
2    train_model #2    1.652845       1.647868  139994266422688
     to_array #0       0.043760       0.039731  139994266422688
     multiply #1       0.029897       0.024791  139994266422688
3    train_model #2    1.298541       1.293026  139994121185768
     to_array #0       0.045227       0.040213  139994121185768
     multiply #1       0.033502       0.030644  139994121185768
4    train_model #2    1.454606       1.450196  139994266422688
     to_array #0       0.046735       0.041521  139994266422688
     multiply #1       0.035175       0.032045  139994266422688
5    train_model #2    1.379844       1.375117  139994121545432
     to_array #0       0.042979       0.038731  139994121545432
     multiply #1       0.039776       0.036961  139994121545432
6    train_model #2    1.470328       1.465981  139994147965584
     to_array #0       0.059414       0.056368  139994147965584
     multiply #1       0.023794       0.020913  139994147965584
7    train_model #2    1.403927       1.399618  139994114391120
     to_array #0       0.040669       0.037654  139994114391120
     multiply #1       0.025973       0.023206  139994114391120
8    train_model #2    1.290563       1.286370  139994266422688
     to_array #0       0.047976       0.043082  139994266422688
     multiply #1       0.028793       0.026014  139994266422688
9    train_model #2    1.557899       1.545445  139994121545432
     to_array #0       0.053481       0.045220  139994121545432
     multiply #1       0.029404       0.026678  139994121545432
10   train_model #2    1.516459       1.511827  139994120640160
     to_array #0       0.100030       0.080075  139994120640160
     multiply #1       0.043438       0.039798  139994120640160
...                         ...            ...              ...
41   train_model #2    1.372318       1.367815  139994085289152
     to_array #0       0.053491       0.050423  139994085289152
     multiply #1       0.040002       0.037197  139994085289152
42   train_model #2    1.813250       1.801895  139993976315800
     to_array #0       0.043776       0.038399  139993976315800
     multiply #1       0.021445       0.018358  139993976315800
43   train_model #2    1.713757       1.709046  139994085289152
     to_array #0       0.085090       0.081801  139994085289152
     multiply #1       0.054966       0.050499  139994085289152
44   train_model #2    1.894864       1.881982  139993976315800
     to_array #0       0.091153       0.087466  139993976315800
     multiply #1       0.042776       0.038372  139993976315800
45   train_model #2    0.981556       0.977580  139994065303872
     to_array #0       0.055165       0.046776  139994065303872
     multiply #1       0.031591       0.027191  139994065303872
46   train_model #2    0.995855       0.991259  139993976315800
     to_array #0       0.038230       0.031463  139993976315800
     multiply #1       0.021815       0.018316  139993976315800
47   train_model #2    1.048226       1.044194  139994085289152
     to_array #0       0.045290       0.040356  139994085289152
     multiply #1       0.020976       0.018173  139994085289152
48   train_model #2    1.156613       1.152424  139993958521208
     to_array #0       0.043842       0.040790  139993958521208
     multiply #1       0.021237       0.018435  139993958521208
49   train_model #2    1.118411       1.109987  139994065302360
     to_array #0       0.040939       0.036036  139994065302360
     multiply #1       0.028280       0.025523  139994065302360
50   train_model #2    1.065028       1.060632  139993976315800
     to_array #0       0.062030       0.054732  139993976315800
     multiply #1       0.037935       0.032852  13999

In [11]:
# for each iter each action show 3 of the slowest methods, based on maximum `ncalls`
pipeline.show_profile_info(per_iter=True, detailed=True, sortby='tottime', limit=3)

ncalls  \
iter action         id                                                           
1    multiply #1    find_in_index::../../../batchflow/batchflow/com...    4096   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
                    _get::../../../batchflow/batchflow/components.p...      64   
     to_array #0    find_in_index::../../../batchflow/batchflow/com...    4096   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
                    <built-in method numpy.where>::../../../batchfl...    4096   
     train_model #2 <method 'run_backward' of 'torch._C._EngineBase...       1   
                    <built-in method conv2d>::/home/tsimfer/anacond...     143   
                    <built-in method batch_norm>::/home/tsimfer/ana...     143   
2    multiply #1    find_in_index::../../../batchflow/batchflow/com...    4096   
                    _get::../../../batchflow/batchflow/components.p...      64   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
     to_array #0    find_in_index::../../../batchflow/batchflow/com...    4096   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
                    __array_interface__::~::0::<built-in method num...      64   
     train_model #2 <method 'run_backward' of 'torch._C._EngineBase...       1   
                    __init__::/home/tsimfer/anaconda3/lib/python3.7...   10843   
                    <built-in method conv2d>::/home/tsimfer/anacond...      20   
3    multiply #1    find_in_index::../../../batchflow/batchflow/com...    4096   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
                    _get::../../../batchflow/batchflow/components.p...      64   
     to_array #0    find_in_index::../../../batchflow/batchflow/com...    4096   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
                    <built-in method numpy.where>::../../../batchfl...    4096   
     train_model #2 <method 'run_backward' of 'torch._C._EngineBase...       1   
                    <built-in method conv2d>::/home/tsimfer/anacond...      20   
                    parse_cpu_trace::/home/tsimfer/anaconda3/lib/py...       1   
4    multiply #1    find_in_index::../../../batchflow/batchflow/com...    4096   
                    _get::../../../batchflow/batchflow/components.p...      64   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
...                                                                        ...   
47   train_model #2 <method 'run_backward' of 'torch._C._EngineBase...       1   
                    parse_cpu_trace::/home/tsimfer/anaconda3/lib/py...       1   
                    <built-in method conv2d>::/home/tsimfer/anacond...      20   
48   multiply #1    find_in_index::../../../batchflow/batchflow/com...    4096   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
                    <built-in method numpy.where>::../../../batchfl...    4096   
     to_array #0    find_in_index::../../../batchflow/batchflow/com...    4096   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
                    <built-in method numpy.where>::../../../batchfl...    4096   
     train_model #2 <method 'run_backward' of 'torch._C._EngineBase...       1   
                    parse_cpu_trace::/home/tsimfer/anaconda3/lib/py...       1   
                    <built-in method conv2d>::/home/tsimfer/anacond...      20   
49   multiply #1    find_in_index::../../../batchflow/batchflow/com...    4096   
                    _get::../../../batchflow/batchflow/components.p...      64   
                    <listcomp>::../../../batchflow/batchflow/compon...      64   
     to_array #0    find_in_index::../../../batchflow/batchflow/com...    4096   
                    <listcomp>::../../../batchflow/bat

# Model profiling

In [12]:
model = pipeline.m('resnet')

There is an `info` property that, unsurprisingly, shows a lot of interesting details regarding model itself or the training process:

In [13]:
model.info


##### Config:
{'benchmark': True,
 'body': {'encoder': {'blocks': {'base': <class 'batchflow.models.torch.blocks.ResBlock'>,
                                 'bottleneck': False,
                                 'downsample': [False, True, True, True],
                                 'filters': [64, 128, 256, 512],
                                 'layout': 'cnacn',
                                 'n_reps': [2, 2, 2, 2],
                                 'se': False},
                      'downsample': {'layout': 'p',
                                     'pool_size': 2,
                                     'pool_strides': 2},
                      'num_stages': 4,
                      'order': ['skip', 'block']}},
 'common': {'data_format': 'channels_first'},
 'decay': None,
 'device': None,
 'head': {'classes': 10,
          'dropout_rate': 0.4,
          'filters': 10,
          'layout': 'Vdf',
          'target_shape': (64,),
          'units': 10},
 'initial_block': {'filters'

As with pipeline, there is a `profile_info` attribute, as well as `show_profile_info` method. Depending on type of the used device (`CPU` or `GPU`)

In [14]:
# one row for every operation inside model; limit at 5 rows
model.show_profile_info(per_iter=False, limit=5)

ncalls              CPU_tottime            \
                               sum    mean  max         sum      mean   
name                                                                    
mkldnn_convolution_backward   1000   20.00   20   38.598603  0.771972   
mkldnn_convolution            1000   20.00   20    6.414861  0.128297   
sqrt                          3038   62.00   62    3.609079  0.073655   
native_batch_norm_backward    1000   20.00   20    2.508889  0.050178   
add_                         10114  202.28  206    1.842055  0.036841   

                                      CPU_cumtime                      \
                                  max         sum      mean       max   
name                                                                    
mkldnn_convolution_backward  1.102136   38.598603  0.771972  1.102136   
mkldnn_convolution           0.219037    6.414861  0.128297  0.219037   
sqrt                         0.117845    3.609079  0.073655  0.117845   
native_batch_norm_backward   0.078185    2.508889  0.050178  0.078185   
add_                         0.056120    1.842055  0.036841  0.056120   

                            CPU_tottime_avg                      
                                        sum      mean       max  
name                                                             
mkldnn_convolution_backward        1.929930  0.038599  0.055107  
mkldnn_convolution                 0.320743  0.006415  0.010952  
sqrt                               0.058211  0.001188  0.001901  
native_batch_norm_backward         0.125444  0.002509  0.003909  
add_                               0.008953  0.000179  0.000272

In [15]:
# for each iteration show 3 of the slowest operations
model.show_profile_info(per_iter=True, limit=3)

ncalls  CPU_tottime  CPU_cumtime  \
iter name                                                            
0    mkldnn_convolution_backward      20     1.102136     1.102136   
     mkldnn_convolution               20     0.116687     0.116687   
     native_batch_norm_backward       20     0.073042     0.073042   
1    mkldnn_convolution_backward      20     0.910873     0.910873   
     mkldnn_convolution               20     0.109726     0.109726   
     sqrt                             62     0.061738     0.061738   
2    mkldnn_convolution_backward      20     0.674379     0.674379   
     mkldnn_convolution               20     0.142355     0.142355   
     sqrt                             62     0.070446     0.070446   
3    mkldnn_convolution_backward      20     0.741461     0.741461   
     mkldnn_convolution               20     0.103317     0.103317   
     sqrt                             62     0.058764     0.058764   
4    mkldnn_convolution_backward      20     0.768156     0.768156   
     mkldnn_convolution               20     0.113721     0.113721   
     sqrt                             62     0.069744     0.069744   
5    mkldnn_convolution_backward      20     0.745379     0.745379   
     mkldnn_convolution               20     0.113744     0.113744   
     sqrt                             62     0.074473     0.074473   
6    mkldnn_convolution_backward      20     0.743273     0.743273   
     mkldnn_convolution               20     0.136254     0.136254   
     sqrt                             62     0.106651     0.106651   
7    mkldnn_convolution_backward      20     0.721489     0.721489   
     mkldnn_convolution               20     0.117488     0.117488   
     sqrt                             62     0.056416     0.056416   
8    mkldnn_convolution_backward      20     0.816239     0.816239   
     mkldnn_convolution               20     0.104785     0.104785   
     sqrt                             62     0.088693     0.088693   
9    mkldnn_convolution_backward      20     0.762722     0.762722   
     mkldnn_convolution               20     0.106693     0.106693   
     sqrt                             62     0.083834     0.083834   
...                                  ...          ...          ...   
40   mkldnn_convolution_backward      20     0.503122     0.503122   
     mkldnn_convolution               20     0.089508     0.089508   
     sqrt                             62     0.066431     0.066431   
41   mkldnn_convolution_backward      20     1.013086     1.013086   
     mkldnn_convolution               20     0.131901     0.131901   
     sqrt                             62     0.090812     0.090812   
42   mkldnn_convolution_backward      20     0.860204     0.860204   
     mkldnn_convolution               20     0.124513     0.124513   
     sqrt                             62     0.117845     0.117845   
43   mkldnn_convolution_backward      20     1.018114     1.018114   
     mkldnn_convolution               20     0.190651     0.190651   
     sqrt                             62     0.090844     0.090844   
44   mkldnn_convolution_backward      20     0.480101     0.480101   
     mkldnn_convolution               20     0.080808     0.080808   
     sqrt                             62     0.072511     0.072511   
45   mkldnn_convolution_backward      20     0.504529     0.504529   
     mkldnn_convolution               20     0.079474     0.079474   
     sqrt                             62     0.066213     0.066213   
46   mkldnn_convolution_backward      20     0.526187     0.526187   
     mkldnn_convolution               20     0.104905     0.104905   
     sqrt                             62     0.061077     0.061077   
47   mkldnn_convolution_backward      20     0.654809     0.654809   
     mkldnn_convolution               20     0.081344     0.081344   
     sqrt                             62     0.063444     0.063444   
48   mkldnn_convolution_backward      20    